In [1]:
import de_core_news_sm
import en_core_web_sm
import portalocker

In [2]:
from subword_nmt.learn_bpe import learn_bpe
from subword_nmt import apply_bpe
from subword_nmt.apply_bpe import BPE
import nltk
import gensim

In [3]:
from torchtext.data.metrics import bleu_score
from torchtext.data.functional import to_map_style_dataset
from torch.utils.data import DataLoader
from torchtext.vocab import build_vocab_from_iterator, vocab
import torchtext.datasets as datasets
from torch.utils.data.distributed import DistributedSampler
import torch.distributed as dist
import torch.multiprocessing as mp
from torch.nn.parallel import DistributedDataParallel as DDP
import torch
import torch.nn as nn
from torch.nn.functional import log_softmax, pad
from torch.optim.lr_scheduler import LambdaLR

In [4]:
from sklearn.utils import shuffle
import pandas as pd
import spacy

In [5]:
import os
from os.path import exists
import json
from collections import Counter, OrderedDict
import math
import copy
import time
import gc
import altair as alt
import matplotlib.pyplot as plt
import GPUtil
import warnings
from entmax import sparsemax, entmax15
warnings.filterwarnings('ignore')
RUN_EXAMPLES = True

In [6]:
from entmax import sparsemax, entmax15

In [7]:
from gensim.models import Word2Vec, KeyedVectors

In [8]:
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:512"

In [9]:
# Some convenience helper functions used throughout the notebook

def is_interactive_notebook():
    return __name__ == '__main__'


def show_example(fn, args=[]):
    if __name__ == '__main__' and RUN_EXAMPLES:
        return fn(*args)


def execute_example(fn, args=[]):
    if __name__ == '__main__' and RUN_EXAMPLES:
        fn(*args)


class DummyOptimizer(torch.optim.Optimizer):
    def __init__(self):
        self.param_groups = [{'lr': 0}]
        None

    def step(self):
        None

    def zero_grad(self, set_to_none=False):
        None


class DummyScheduler:
    def step(self):
        None

In [10]:
# plt.plot([2.8922460307237907, 2.194109773455518, 2.0563710700215725, 1.979828157081014], label='Softmax Attention')
# plt.plot([4.57104134212686, 3.9963503417950355, 3.7907971491658072, 3.6298673703196926], label='Rectified Linear Attention')
# plt.plot([3.2588626102080673, 2.5048274199231493, 2.317883473525828, 2.2120773215414973], label='Sparsemax Attention')

# plt.xticks(range(4))
# plt.xlabel('Epochs')
# plt.ylabel('Loss')
# plt.title('Train Loss Over Time')
# plt.legend()
# plt.show()

In [11]:
torch.cuda.empty_cache()

In [12]:
# max_train, max_val, max_test = 2977999, 18719, 19151
# max_train, max_val, max_test = 50000, 15000, 15000
# max_train, max_val, max_test = 1000, 500, 500

In [13]:
# './models/vi_to_en_complete_softmax_3.pt'
# './models/vi_to_en_complete_rela_3.pt'
# './models/vi_to_en_complete_sparsemax_3.pt'

In [14]:
# a = torch.load('./models/vi_to_en_complete_softmax_3.pt')

In [15]:
# a['validation_epoch_loss']

In [16]:
# b = torch.load('./models/vi_to_en_BPE_complete_softmax_3.pt')

In [17]:
# b['score']

In [18]:
# c = torch.load('./models/vi_to_en_complete_rela_3.pt')

In [19]:
# c['validation_epoch_loss']

In [20]:
# d = torch.load('./models/vi_to_en_BPE_complete_rela_3.pt')

In [21]:
# d['score']

In [22]:
# e = torch.load('./models/vi_to_en_complete_sparsemax_3.pt')

In [23]:
# e['validation_epoch_loss']

In [24]:
# f = torch.load('./models/vi_to_en_BPE_complete_sparsemax_3.pt')

In [25]:
# f['score']

# Part 1: Model Architecture

# Model Architecture


Most competitive neural sequence transduction models have an
encoder-decoder structure
[(cite)](https://arxiv.org/abs/1409.0473). Here, the encoder maps an
input sequence of symbol representations $(x_1, ..., x_n)$ to a
sequence of continuous representations $\mathbf{z} = (z_1, ...,
z_n)$. Given $\mathbf{z}$, the decoder then generates an output
sequence $(y_1,...,y_m)$ of symbols one element at a time. At each
step the model is auto-regressive
[(cite)](https://arxiv.org/abs/1308.0850), consuming the previously
generated symbols as additional input when generating the next.

In [26]:
class EncoderDecoder(nn.Module):
    '''
    A standard Encoder-Decoder architecture. Base for this and many
    other models.
    '''

    def __init__(self, encoder, decoder, src_embed, tgt_embed, generator):
        super(EncoderDecoder, self).__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.src_embed = src_embed
        self.tgt_embed = tgt_embed
        self.generator = generator

    def forward(self, src, tgt, src_mask, tgt_mask):
        'Take in and process masked src and target sequences.'
        src_encode = self.encode(src, src_mask)
        return self.decode(src_encode, src_mask, tgt, tgt_mask)

    def encode(self, src, src_mask):
        src_embed = self.src_embed(src)
        return self.encoder(src_embed, src_mask)

    def decode(self, memory, src_mask, tgt, tgt_mask):
        tgt_embed = self.tgt_embed(tgt)
        return self.decoder(tgt_embed, memory, src_mask, tgt_mask)

In [27]:
class Generator(nn.Module):
    'Define standard linear + softmax generation step.'

    def __init__(self, d_model, vocab):
        super(Generator, self).__init__()
        self.proj = nn.Linear(d_model, vocab)

    def forward(self, x):
        return log_softmax(self.proj(x), dim=-1)


The Transformer follows this overall architecture using stacked
self-attention and point-wise, fully connected layers for both the
encoder and decoder, shown in the left and right halves of Figure 1,
respectively.

![](images/ModalNet-21.png)

## Encoder and Decoder Stacks

### Encoder

The encoder is composed of a stack of $N=6$ identical layers.

In [28]:
def clones(module, N):
    'Produce N identical layers.'
    return nn.ModuleList([copy.deepcopy(module) for _ in range(N)])

In [29]:
class Encoder(nn.Module):
    'Core encoder is a stack of N layers'

    def __init__(self, layer, N):
        super(Encoder, self).__init__()
        self.layers = clones(layer, N)
        self.norm = LayerNorm(layer.size)

    def forward(self, x, mask):
        'Pass the input (and mask) through each layer in turn.'
        for layer in self.layers:
            x = layer(x, mask)
        return self.norm(x)


We employ a residual connection
[(cite)](https://arxiv.org/abs/1512.03385) around each of the two
sub-layers, followed by layer normalization
[(cite)](https://arxiv.org/abs/1607.06450).

In [30]:
class LayerNorm(nn.Module):
    'Construct a layernorm module (See citation for details).'

    def __init__(self, features, eps=1e-6):
        super(LayerNorm, self).__init__()
        self.a_2 = nn.Parameter(torch.ones(features))
        self.b_2 = nn.Parameter(torch.zeros(features))
        self.eps = eps

    def forward(self, x):
        mean = x.mean(-1, keepdim=True)
        std = x.std(-1, keepdim=True)
        return self.a_2 * (x - mean) / (std + self.eps) + self.b_2


That is, the output of each sub-layer is $\mathrm{LayerNorm}(x +
\mathrm{Sublayer}(x))$, where $\mathrm{Sublayer}(x)$ is the function
implemented by the sub-layer itself.  We apply dropout
[(cite)](http://jmlr.org/papers/v15/srivastava14a.html) to the
output of each sub-layer, before it is added to the sub-layer input
and normalized.

To facilitate these residual connections, all sub-layers in the
model, as well as the embedding layers, produce outputs of dimension
$d_{\text{model}}=512$.

In [31]:
class SublayerConnection(nn.Module):
    '''
    A residual connection followed by a layer norm.
    Note for code simplicity the norm is first as opposed to last.
    '''

    def __init__(self, size, dropout):
        super(SublayerConnection, self).__init__()
        self.norm = LayerNorm(size)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, sublayer):
        'Apply residual connection to any sublayer with the same size.'
        return x + self.dropout(sublayer(self.norm(x)))


Each layer has two sub-layers. The first is a multi-head
self-attention mechanism, and the second is a simple, position-wise
fully connected feed-forward network.

In [32]:
class EncoderLayer(nn.Module):
    'Encoder is made up of self-attn and feed forward (defined below)'

    def __init__(self, size, self_attn, feed_forward, dropout):
        super(EncoderLayer, self).__init__()
        self.self_attn = self_attn
        self.feed_forward = feed_forward
        # creates the two 'add & norm' found in the figure
        self.sublayer = clones(SublayerConnection(size, dropout), 2)
        self.size = size

    def forward(self, x, mask):
        'Follow Figure 1 (left) for connections.'
        # first, feed the multi-head attention to the sublayer 'add & norm'
        x = self.sublayer[0](x, lambda x: self.self_attn(x, x, x, mask))
        # next feed that output along with feed forward into the sublayer 'add & norm'
        return self.sublayer[1](x, self.feed_forward)

### Decoder

The decoder is also composed of a stack of $N=6$ identical layers.


In [33]:
class Decoder(nn.Module):
    'Generic N layer decoder with masking.'

    def __init__(self, layer, N):
        super(Decoder, self).__init__()
        self.layers = clones(layer, N)
        self.norm = LayerNorm(layer.size)

    def forward(self, x, memory, src_mask, tgt_mask):
        for layer in self.layers:
            x = layer(x, memory, src_mask, tgt_mask)
        return self.norm(x)


In addition to the two sub-layers in each encoder layer, the decoder
inserts a third sub-layer, which performs multi-head attention over
the output of the encoder stack.  Similar to the encoder, we employ
residual connections around each of the sub-layers, followed by
layer normalization.

In [34]:
class DecoderLayer(nn.Module):
    'Decoder is made of self-attn, src-attn, and feed forward (defined below)'

    def __init__(self, size, self_attn, src_attn, feed_forward, dropout):
        super(DecoderLayer, self).__init__()
        self.size = size
        self.self_attn = self_attn
        self.src_attn = src_attn
        self.feed_forward = feed_forward
        # 3 'add & norm'
        self.sublayer = clones(SublayerConnection(size, dropout), 3)

    def forward(self, x, memory, src_mask, tgt_mask):
        'Follow Figure 1 (right) for connections.'
        m = memory
        # do self attention first, then do cross attention using memory
        x = self.sublayer[0](x, lambda x: self.self_attn(x, x, x, tgt_mask))
        x = self.sublayer[1](x, lambda x: self.src_attn(x, m, m, src_mask))
        return self.sublayer[2](x, self.feed_forward)


We also modify the self-attention sub-layer in the decoder stack to
prevent positions from attending to subsequent positions.  This
masking, combined with fact that the output embeddings are offset by
one position, ensures that the predictions for position $i$ can
depend only on the known outputs at positions less than $i$.

In [35]:
def subsequent_mask(size):
    'Mask out subsequent positions.'
    attn_shape = (1, size, size)
    subsequent_mask = torch.triu(torch.ones(attn_shape), diagonal=1).type(torch.uint8)
    return subsequent_mask == 0


> Below the attention mask shows the position each tgt word (row) is
> allowed to look at (column). Words are blocked for attending to
> future words during training.

In [36]:
# def example_mask():
#     LS_data = pd.concat(
#         [
#             pd.DataFrame(
#                 {
#                     'Subsequent Mask': subsequent_mask(20)[0][x, y].flatten(),
#                     'Window': y,
#                     'Masking': x,
#                 }
#             )
#             for y in range(20)
#             for x in range(20)
#         ]
#     )

#     return (
#         alt.Chart(LS_data)
#         .mark_rect()
#         .properties(height=250, width=250)
#         .encode(
#             alt.X('Window:O'),
#             alt.Y('Masking:O'),
#             alt.Color('Subsequent Mask:Q', scale=alt.Scale(scheme='viridis')),
#         )
#         .interactive()
#     )


# show_example(example_mask)

### Attention

An attention function can be described as mapping a query and a set
of key-value pairs to an output, where the query, keys, values, and
output are all vectors.  The output is computed as a weighted sum of
the values, where the weight assigned to each value is computed by a
compatibility function of the query with the corresponding key.

We call our particular attention 'Scaled Dot-Product Attention'.
The input consists of queries and keys of dimension $d_k$, and
values of dimension $d_v$.  We compute the dot products of the query
with all keys, divide each by $\sqrt{d_k}$, and apply a softmax
function to obtain the weights on the values.



![](images/ModalNet-19.png)


In practice, we compute the attention function on a set of queries
simultaneously, packed together into a matrix $Q$.  The keys and
values are also packed together into matrices $K$ and $V$.  We
compute the matrix of outputs as:

$$
   \mathrm{Attention}(Q, K, V) = \mathrm{softmax}(\frac{QK^T}{\sqrt{d_k}})V
$$

In [37]:
def attention(query, key, value, attention_method='softmax', mask=None, dropout=None, norm=None):
    "Compute Scaled Dot Product Attention"
    d_k = query.size(-1)
    scores = torch.matmul(query, key.transpose(-2, -1)) / math.sqrt(d_k)
    if mask is not None:
        scores = scores.masked_fill(mask == 0, -1e9)
    
    if attention_method == 'softmax':
        p_attn = scores.softmax(dim=-1)
    elif attention_method == 'rela':
        p_attn = scores.relu() 
    elif attention_method == 'sparsemax':
        p_attn = sparsemax(scores, dim=-1)
    elif attention_method == 'entmax':
        p_attn = entmax15(scores, dim=-1)
    else:
        pass
        
    if dropout is not None:
        p_attn = dropout(p_attn)
            
    if attention_method == 'softmax':
        return torch.matmul(p_attn, value), p_attn
    elif attention_method == 'rela':
        return norm(torch.matmul(p_attn, value)), p_attn
    elif attention_method == 'sparsemax':
        return torch.matmul(p_attn, value), p_attn
    elif attention_method == 'entmax':
        return torch.matmul(p_attn, value), p_attn
    else:
        pass
    


The two most commonly used attention functions are additive
attention [(cite)](https://arxiv.org/abs/1409.0473), and dot-product
(multiplicative) attention.  Dot-product attention is identical to
our algorithm, except for the scaling factor of
$\frac{1}{\sqrt{d_k}}$. Additive attention computes the
compatibility function using a feed-forward network with a single
hidden layer.  While the two are similar in theoretical complexity,
dot-product attention is much faster and more space-efficient in
practice, since it can be implemented using highly optimized matrix
multiplication code.


While for small values of $d_k$ the two mechanisms perform
similarly, additive attention outperforms dot product attention
without scaling for larger values of $d_k$
[(cite)](https://arxiv.org/abs/1703.03906). We suspect that for
large values of $d_k$, the dot products grow large in magnitude,
pushing the softmax function into regions where it has extremely
small gradients (To illustrate why the dot products get large,
assume that the components of $q$ and $k$ are independent random
variables with mean $0$ and variance $1$.  Then their dot product,
$q \cdot k = \sum_{i=1}^{d_k} q_ik_i$, has mean $0$ and variance
$d_k$.). To counteract this effect, we scale the dot products by
$\frac{1}{\sqrt{d_k}}$.



![](images/ModalNet-20.png)


Multi-head attention allows the model to jointly attend to
information from different representation subspaces at different
positions. With a single attention head, averaging inhibits this.

$$
\mathrm{MultiHead}(Q, K, V) =
    \mathrm{Concat}(\mathrm{head_1}, ..., \mathrm{head_h})W^O \\
    \text{where}~\mathrm{head_i} = \mathrm{Attention}(QW^Q_i, KW^K_i, VW^V_i)
$$

Where the projections are parameter matrices $W^Q_i \in
\mathbb{R}^{d_{\text{model}} \times d_k}$, $W^K_i \in
\mathbb{R}^{d_{\text{model}} \times d_k}$, $W^V_i \in
\mathbb{R}^{d_{\text{model}} \times d_v}$ and $W^O \in
\mathbb{R}^{hd_v \times d_{\text{model}}}$.

In this work we employ $h=8$ parallel attention layers, or
heads. For each of these we use $d_k=d_v=d_{\text{model}}/h=64$. Due
to the reduced dimension of each head, the total computational cost
is similar to that of single-head attention with full
dimensionality.

In [38]:
class MultiHeadedAttention(nn.Module):
    def __init__(self, h, d_model, dropout=0.1, attention_method='softmax'):
        'Take in model size and number of heads.'
        super(MultiHeadedAttention, self).__init__()
        assert d_model % h == 0
        # We assume d_v always equals d_k
        self.d_k = d_model // h
        self.h = h
        self.linears = clones(nn.Linear(d_model, d_model), 4)
        self.attn = None
        self.attention_method = attention_method
        self.dropout = nn.Dropout(p=dropout)
        self.norm = LayerNorm(int(d_model/h))

    def forward(self, query, key, value, mask=None):
        'Implements Figure 2'
        if mask is not None:
            # Same mask applied to all h heads.
            mask = mask.unsqueeze(1)
        nbatches = query.size(0)

        # 1) Do all the linear projections in batch from d_model => h x d_k
        query, key, value = [
            lin(x).view(nbatches, -1, self.h, self.d_k).transpose(1, 2)
            for lin, x in zip(self.linears, (query, key, value))
        ]

        # 2) Apply attention on all the projected vectors in batch.
        x, self.attn = attention(query, key, value, self.attention_method, mask=mask, dropout=self.dropout, norm=self.norm)

        # 3) 'Concat' using a view and apply a final linear.
        x = x.transpose(1, 2).contiguous().view(nbatches, -1, self.h * self.d_k)
        del query
        del key
        del value
        
        return self.linears[-1](x)

### Applications of Attention in our Model

The Transformer uses multi-head attention in three different ways:
1) In 'encoder-decoder attention' layers, the queries come from the
previous decoder layer, and the memory keys and values come from the
output of the encoder.  This allows every position in the decoder to
attend over all positions in the input sequence.  This mimics the
typical encoder-decoder attention mechanisms in sequence-to-sequence
models such as [(cite)](https://arxiv.org/abs/1609.08144).


2) The encoder contains self-attention layers.  In a self-attention
layer all of the keys, values and queries come from the same place,
in this case, the output of the previous layer in the encoder.  Each
position in the encoder can attend to all positions in the previous
layer of the encoder.


3) Similarly, self-attention layers in the decoder allow each
position in the decoder to attend to all positions in the decoder up
to and including that position.  We need to prevent leftward
information flow in the decoder to preserve the auto-regressive
property.  We implement this inside of scaled dot-product attention
by masking out (setting to $-\infty$) all values in the input of the
softmax which correspond to illegal connections.

## Position-wise Feed-Forward Networks

In addition to attention sub-layers, each of the layers in our
encoder and decoder contains a fully connected feed-forward network,
which is applied to each position separately and identically.  This
consists of two linear transformations with a ReLU activation in
between.

$$\mathrm{FFN}(x)=\max(0, xW_1 + b_1) W_2 + b_2$$

While the linear transformations are the same across different
positions, they use different parameters from layer to
layer. Another way of describing this is as two convolutions with
kernel size 1.  The dimensionality of input and output is
$d_{\text{model}}=512$, and the inner-layer has dimensionality
$d_{ff}=2048$.

In [39]:
class PositionwiseFeedForward(nn.Module):
    'Implements FFN equation.'

    def __init__(self, d_model, d_ff, dropout=0.1):
        super(PositionwiseFeedForward, self).__init__()
        self.w_1 = nn.Linear(d_model, d_ff)
        self.w_2 = nn.Linear(d_ff, d_model)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        x = self.w_1(x).relu()
        x = self.dropout(x)
        return self.w_2(x)

## Embeddings and Softmax

Similarly to other sequence transduction models, we use learned
embeddings to convert the input tokens and output tokens to vectors
of dimension $d_{\text{model}}$.  We also use the usual learned
linear transformation and softmax function to convert the decoder
output to predicted next-token probabilities.  In our model, we
share the same weight matrix between the two embedding layers and
the pre-softmax linear transformation, similar to
[(cite)](https://arxiv.org/abs/1608.05859). In the embedding layers,
we multiply those weights by $\sqrt{d_{\text{model}}}$.

In [40]:
class Embeddings(nn.Module):
    def __init__(self, d_model, vocab):
        super(Embeddings, self).__init__()
        self.lut = nn.Embedding(vocab, d_model)
        self.d_model = d_model

    def forward(self, x):
        return self.lut(x) * math.sqrt(self.d_model)

## Positional Encoding

Since our model contains no recurrence and no convolution, in order
for the model to make use of the order of the sequence, we must
inject some information about the relative or absolute position of
the tokens in the sequence.  To this end, we add 'positional
encodings' to the input embeddings at the bottoms of the encoder and
decoder stacks.  The positional encodings have the same dimension
$d_{\text{model}}$ as the embeddings, so that the two can be summed.
There are many choices of positional encodings, learned and fixed
[(cite)](https://arxiv.org/pdf/1705.03122.pdf).

In this work, we use sine and cosine functions of different frequencies:

$$PE_{(pos,2i)} = \sin(pos / 10000^{2i/d_{\text{model}}})$$

$$PE_{(pos,2i+1)} = \cos(pos / 10000^{2i/d_{\text{model}}})$$

where $pos$ is the position and $i$ is the dimension.  That is, each
dimension of the positional encoding corresponds to a sinusoid.  The
wavelengths form a geometric progression from $2\pi$ to $10000 \cdot
2\pi$.  We chose this function because we hypothesized it would
allow the model to easily learn to attend by relative positions,
since for any fixed offset $k$, $PE_{pos+k}$ can be represented as a
linear function of $PE_{pos}$.

In addition, we apply dropout to the sums of the embeddings and the
positional encodings in both the encoder and decoder stacks.  For
the base model, we use a rate of $P_{drop}=0.1$.



In [41]:
class PositionalEncoding(nn.Module):
    'Implement the PE function.'

    def __init__(self, d_model, dropout, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)

        # Compute the positional encodings once in log space.
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len).unsqueeze(1)
        div_term = torch.exp(
            torch.arange(0, d_model, 2) * -(math.log(10000.0) / d_model)
        )
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:, : x.size(1)].requires_grad_(False)
        return self.dropout(x)


> Below the positional encoding will add in a sine wave based on
> position. The frequency and offset of the wave is different for
> each dimension.

In [42]:
# def example_positional():
#     pe = PositionalEncoding(20, 0)
#     y = pe.forward(torch.zeros(1, 100, 20))

#     data = pd.concat(
#         [
#             pd.DataFrame(
#                 {
#                     'embedding': y[0, :, dim],
#                     'dimension': dim,
#                     'position': list(range(100)),
#                 }
#             )
#             for dim in [4, 5, 6, 7]
#         ]
#     )

#     return (
#         alt.Chart(data)
#         .mark_line()
#         .properties(width=800)
#         .encode(x='position', y='embedding', color='dimension:N')
#         .interactive()
#     )


# show_example(example_positional)


We also experimented with using learned positional embeddings
[(cite)](https://arxiv.org/pdf/1705.03122.pdf) instead, and found
that the two versions produced nearly identical results.  We chose
the sinusoidal version because it may allow the model to extrapolate
to sequence lengths longer than the ones encountered during
training.

## Full Model

> Here we define a function from hyperparameters to a full model.

In [43]:
def make_model(src_dim, tgt_dim, N=6, d_model=512, d_ff=2048, h=8, dropout=0.1, attention_method='softmax'):
    'Helper: Construct a model from hyperparameters.'
    c = copy.deepcopy
    attn = MultiHeadedAttention(h=h, d_model=d_model, attention_method=attention_method)
    ff = PositionwiseFeedForward(d_model, d_ff, dropout)
    position = PositionalEncoding(d_model, dropout)
    model = EncoderDecoder(
        Encoder(EncoderLayer(d_model, c(attn), c(ff), dropout), N),
        Decoder(DecoderLayer(d_model, c(attn), c(attn), c(ff), dropout), N),
        nn.Sequential(Embeddings(d_model, src_dim), c(position)),
        nn.Sequential(Embeddings(d_model, tgt_dim), c(position)),
        Generator(d_model, tgt_dim),
    )

    # This was important from their code.
    # Initialize parameters with Glorot / fan_avg.
    for p in model.parameters():
        if p.dim() > 1:
            nn.init.xavier_uniform_(p)
    return model

## Inference:

> Here we make a forward step to generate a prediction of the
model. We try to use our transformer to memorize the input. As you
will see the output is randomly generated due to the fact that the
model is not trained yet. In the next tutorial we will build the
training function and try to train our model to memorize the numbers
from 1 to 10.

In [44]:
# def inference_test():
#     test_model = make_model(11, 11, 2)
#     test_model.eval()
#     src = torch.LongTensor([[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]])
#     src_mask = torch.ones(1, 1, 10)

#     memory = test_model.encode(src, src_mask)
#     ys = torch.zeros(1, 1).type_as(src)

#     for i in range(9):
#         out = test_model.decode(
#             memory, src_mask, ys, subsequent_mask(ys.size(1)).type_as(src.data)
#         )
#         prob = test_model.generator(out[:, -1])
#         _, next_word = torch.max(prob, dim=1)
#         next_word = next_word.data[0]
#         ys = torch.cat(
#             [ys, torch.empty(1, 1).type_as(src.data).fill_(next_word)], dim=1
#         )

#     print('Example Untrained Model Prediction:', ys)


# def run_tests():
#     for _ in range(10):
#         inference_test()


# show_example(run_tests)

# Part 2: Model Training

# Training

This section describes the training regime for our models.


> We stop for a quick interlude to introduce some of the tools
> needed to train a standard encoder decoder model. First we define a
> batch object that holds the src and target sentences for training,
> as well as constructing the masks.

## Batches and Masking

In [45]:
class Batch:
    '''Object for holding a batch of data with mask during training.'''

    def __init__(self, src, tgt=None, pad=2):  # 2 = <blank>
        self.src = src
        self.src_mask = (src != pad).unsqueeze(-2)
        if tgt is not None:
            self.tgt = tgt[:, :-1]
            self.tgt_y = tgt[:, 1:]
            self.tgt_mask = self.make_std_mask(self.tgt, pad)
            self.ntokens = (self.tgt_y != pad).data.sum()

    @staticmethod
    def make_std_mask(tgt, pad):
        'Create a mask to hide padding and future words.'
        tgt_mask = (tgt != pad).unsqueeze(-2)
        tgt_mask = tgt_mask & subsequent_mask(tgt.size(-1)).type_as(tgt_mask.data)
        return tgt_mask


> Next we create a generic training and scoring function to keep
> track of loss. We pass in a generic loss compute function that
> also handles parameter updates.

## Training Loop

In [46]:
def run_epoch(
    data_iter,
    print_each,
    model,
    loss_compute,
    optimizer,
    scheduler,
    mode='train',
    accum_iter=1,
):
    '''Train a single epoch'''
    start = time.time()
    total_tokens = 0
    total_loss = 0
    tokens = 0
    n_accum = 0
    for i, batch in enumerate(data_iter):
        out = model.forward(batch.src, batch.tgt, batch.src_mask, batch.tgt_mask)
        loss, loss_node = loss_compute(out, batch.tgt_y, batch.ntokens)

        if mode == 'train' or mode == 'train+log':
            loss_node.backward()
            
            if i % accum_iter == 0:
                optimizer.step()
                optimizer.zero_grad(set_to_none=True)
                n_accum += 1
            scheduler.step()

        total_loss += loss
        total_tokens += batch.ntokens
        tokens += batch.ntokens
        
        
        if i % print_each == 0:
            elapsed = time.time() - start
            if mode == 'train' or mode == 'train+log':
                lr = optimizer.param_groups[0]['lr']
                print(
                    (
                        'Epoch Step: %6d | Accumulation Step: %3d | Loss: %6.2f '
                        + '| Tokens / Sec: %7.1f | Learning Rate: %6.1e'
                    )
                    % (i, n_accum, loss / batch.ntokens, tokens / elapsed, lr)
                )
                start = time.time()
                tokens = 0
                
            elif mode == 'eval':
                    print(
                    (
                        'Epoch Step: %6d | Tokens / Sec: %7.1f'
                    )
                        
                    % (i, tokens / elapsed)
                    )
        
                        
        del loss
        del loss_node
        
    print('\nAverage Epoch Loss: {:.2f}'.format(total_loss / total_tokens))

    return total_loss.item() / total_tokens.item()

## Training Data and Batching

We trained on the standard WMT 2014 English-German dataset
consisting of about 4.5 million sentence pairs.  Sentences were
encoded using byte-pair encoding, which has a shared source-target
vocabulary of about 37000 tokens. For English-French, we used the
significantly larger WMT 2014 English-French dataset consisting of
36M sentences and split tokens into a 32000 word-piece vocabulary.


Sentence pairs were batched together by approximate sequence length.
Each training batch contained a set of sentence pairs containing
approximately 25000 source tokens and 25000 target tokens.

## Hardware and Schedule

We trained our models on one machine with 8 NVIDIA P100 GPUs.  For
our base models using the hyperparameters described throughout the
paper, each training step took about 0.4 seconds.  We trained the
base models for a total of 100,000 steps or 12 hours. For our big
models, step time was 1.0 seconds.  The big models were trained for
300,000 steps (3.5 days).

## Optimizer

We used the Adam optimizer [(cite)](https://arxiv.org/abs/1412.6980)
with $\beta_1=0.9$, $\beta_2=0.98$ and $\epsilon=10^{-9}$.  We
varied the learning rate over the course of training, according to
the formula:

$$
lrate = d_{\text{model}}^{-0.5} \cdot
  \min({step\_num}^{-0.5},
    {step\_num} \cdot {warmup\_steps}^{-1.5})
$$

This corresponds to increasing the learning rate linearly for the
first $warmup\_steps$ training steps, and decreasing it thereafter
proportionally to the inverse square root of the step number.  We
used $warmup\_steps=4000$.


> Note: This part is very important. Need to train with this setup
> of the model.


> Example of the curves of this model for different model sizes and
> for optimization hyperparameters.

In [47]:
def rate(step, model_size, factor, warmup):
    '''
    we have to default the step to 1 for LambdaLR function
    to avoid zero raising to negative power.
    '''
    if step == 0:
        step = 1
    return factor * (
        model_size ** (-0.5) * min(step ** (-0.5), step * warmup ** (-1.5))
    )

## Regularization

### Label Smoothing

During training, we employed label smoothing of value
$\epsilon_{ls}=0.1$ [(cite)](https://arxiv.org/abs/1512.00567).
This hurts perplexity, as the model learns to be more unsure, but
improves accuracy and BLEU score.


> We implement label smoothing using the KL div loss. Instead of
> using a one-hot target distribution, we create a distribution that
> has `confidence` of the correct word and the rest of the
> `smoothing` mass distributed throughout the vocabulary.

In [48]:
class LabelSmoothing(nn.Module):
    'Implement label smoothing.'

    def __init__(self, size, padding_idx, smoothing=0.0):
        super(LabelSmoothing, self).__init__()
        self.criterion = nn.KLDivLoss(reduction='sum')
        self.padding_idx = padding_idx
        self.confidence = 1.0 - smoothing
        self.smoothing = smoothing
        self.size = size
        self.true_dist = None

    def forward(self, x, target):
        assert x.size(1) == self.size
        true_dist = x.data.clone()
        true_dist.fill_(self.smoothing / (self.size - 2))
        true_dist.scatter_(1, target.data.unsqueeze(1), self.confidence)
        true_dist[:, self.padding_idx] = 0
        mask = torch.nonzero(target.data == self.padding_idx)
        if mask.dim() > 0:
            true_dist.index_fill_(0, mask.squeeze(), 0.0)
        self.true_dist = true_dist
        return self.criterion(x, true_dist.clone().detach())


> Here we can see an example of how the mass is distributed to the
> words based on confidence.

In [49]:
# # Example of label smoothing.

# def example_label_smoothing():
#     crit = LabelSmoothing(5, 0, 0.4)
#     predict = torch.FloatTensor(
#         [
#             [0, 0.2, 0.7, 0.1, 0],
#             [0, 0.2, 0.7, 0.1, 0],
#             [0, 0.2, 0.7, 0.1, 0],
#             [0, 0.2, 0.7, 0.1, 0],
#             [0, 0.2, 0.7, 0.1, 0],
#         ]
#     )
#     crit(x=predict.log(), target=torch.LongTensor([2, 1, 0, 3, 3]))
#     LS_data = pd.concat(
#         [
#             pd.DataFrame(
#                 {
#                     'target distribution': crit.true_dist[x, y].flatten(),
#                     'columns': y,
#                     'rows': x,
#                 }
#             )
#             for y in range(5)
#             for x in range(5)
#         ]
#     )

#     return (
#         alt.Chart(LS_data)
#         .mark_rect(color='Blue', opacity=1)
#         .properties(height=200, width=200)
#         .encode(
#             alt.X('columns:O', title=None),
#             alt.Y('rows:O', title=None),
#             alt.Color(
#                 'target distribution:Q', scale=alt.Scale(scheme='viridis')
#             ),
#         )
#         .interactive()
#     )


# show_example(example_label_smoothing)


> Label smoothing actually starts to penalize the model if it gets
> very confident about a given choice.

In [50]:
# def loss(x, crit):
#     d = x + 3 * 1
#     predict = torch.FloatTensor([[0, x / d, 1 / d, 1 / d, 1 / d]])
#     return crit(predict.log(), torch.LongTensor([1])).data


# def penalization_visualization():
#     crit = LabelSmoothing(5, 0, 0.1)
#     loss_data = pd.DataFrame(
#         {
#             'Loss': [loss(x, crit) for x in range(1, 100)],
#             'Steps': list(range(99)),
#         }
#     ).astype('float')

#     return (
#         alt.Chart(loss_data)
#         .mark_line()
#         .properties(width=350)
#         .encode(
#             x='Steps',
#             y='Loss',
#         )
#         .interactive()
#     )


# show_example(penalization_visualization)

## Loss Computation

In [51]:
class SimpleLossCompute:
    'A simple loss compute and train function.'

    def __init__(self, generator, criterion):
        self.generator = generator
        self.criterion = criterion

    def __call__(self, x, y, norm):
        x = self.generator(x)
        sloss = (
            self.criterion(x.contiguous().view(-1, x.size(-1)), y.contiguous().view(-1))
            / norm
        )
        return sloss.data * norm, sloss

## Greedy Decoding

> This code predicts a translation using greedy decoding for simplicity.

In [52]:
def greedy_decode(model, src, src_mask, max_len, start_symbol):
    memory = model.encode(src, src_mask)
    ys = torch.zeros(1, 1).fill_(start_symbol).type_as(src.data)
    for i in range(max_len - 1):
        out = model.decode(
            memory, src_mask, ys, subsequent_mask(ys.size(1)).type_as(src.data)
        )
        prob = model.generator(out[:, -1])
        _, next_word = torch.max(prob, dim=1)
        next_word = next_word.data[0]
        ys = torch.cat(
            [ys, torch.zeros(1, 1).type_as(src.data).fill_(next_word)], dim=1
        )
    return ys

# Part 3: A Real World Example

> Now we consider a real-world example using the Multi30k
> German-English Translation task. This task is much smaller than
> the WMT task considered in the paper, but it illustrates the whole
> system. We also show how to use multi-gpu processing to make it
> really fast.

## Data Loading

> We will load the dataset using torchtext and spacy for
> tokenization.

In [53]:
def tokenize(text, tokenizer):
    return [tok for tok in tokenizer.tokenizer(text)]
def yield_tokens(data_iter, tokenizer, index):
    for from_to_tuple in data_iter:
        yield tokenizer(from_to_tuple[index])

# Custom Data

In [54]:
class tokenizer:
    def __init__(self):
        self.none = None

    def tokenizer(self, sentence):
        return sentence.split()

In [55]:
class tokenizer_BPE:
    def __init__(self, src=True):
        self.none = None
        if src:
            self.bpe = BPE(open('./PhoMT/BPE/bp_vi.txt', encoding='utf-8'))
        else:
            self.bpe = BPE(open('./PhoMT/BPE/bp_en.txt', encoding='utf-8'))
            
    def tokenizer(self, sentence):
        return self.bpe.segment_tokens(gensim.utils.simple_preprocess(sentence))

In [56]:
def getVocab(counter):
    # define special symbols and indices
    BOS_IDX, EOS_IDX, PAD_IDX, UNK_IDX = 0, 1, 2, 3
    # make sure indices align within vocabulary
    special_symbols = ['<s>', '</s>', '<blank>', '<unk>']

    sorted_by_freq_tuples = sorted(counter.items(), key=lambda x: x[1], reverse=True)
    ordered_dict = OrderedDict(sorted_by_freq_tuples)
    vocab_ = vocab(ordered_dict, min_freq=1, specials=special_symbols)
    # tell it vocab what to return when token not found
    vocab_.set_default_index(UNK_IDX)
    
    return vocab_

In [57]:
def get_Vocab_Tokenizer(max_train, max_val, max_test, USE_BPE=False):
    vi_train_path = './PhoMT/tokenization/train/train.vi'
    en_train_path = './PhoMT/tokenization/train/train.en'

    vi_val_path = './PhoMT/tokenization/dev/dev.vi'
    en_val_path = './PhoMT/tokenization/dev/dev.en'

    vi_test_path = './PhoMT/tokenization/test/test.vi'
    en_test_path = './PhoMT/tokenization/test/test.en'
    
    
    train_vi, train_en = [], []
    with open(vi_train_path, encoding='utf-8') as f:
        for line in f.readlines():
            train_vi.append(line.replace('\n', ''))
    with open(en_train_path, encoding='utf-8') as f:
        for line in f.readlines():
            train_en.append(line.replace('\n', ''))
        
    val_vi, val_en = [], []
    with open(vi_val_path, encoding='utf-8') as f:
        for line in f.readlines():
            val_vi.append(line.replace('\n', ''))
    with open(en_val_path, encoding='utf-8') as f:
        for line in f.readlines():
            val_en.append(line.replace('\n', ''))     
    
    test_vi, test_en = [], []
    with open(vi_test_path, encoding='utf-8') as f:
        for line in f.readlines():
            test_vi.append(line.replace('\n', ''))
    with open(en_test_path, encoding='utf-8') as f:
        for line in f.readlines():
            test_en.append(line.replace('\n', ''))  
            
    train_vi, train_en = shuffle(train_vi, train_en)
    val_vi, val_en = shuffle(val_vi, val_en)
    test_vi, test_en = shuffle(test_vi, test_en)
    
    train_vi, train_en = train_vi[:max_train], train_en[:max_train]
    val_vi, val_en = val_vi[:max_val], val_en[:max_val]
    test_vi, test_en = test_vi[:max_test], test_en[:max_test]
    

    train_iter = list(zip(train_vi, train_en))
    val_iter = list(zip(val_vi, val_en))
    test_iter = list(zip(test_vi, test_en))
    
    if not USE_BPE:
        temp_vi, temp_en = [], []
        for vi, en in train_iter:
            for token in vi.split():
                temp_vi.append(token)
            for token in en.split():
                temp_en.append(token)

        train_vi_counter = Counter(temp_vi)
        train_en_counter = Counter(temp_en)
    else:
        tokenized_corpus_en = [gensim.utils.simple_preprocess(text) for text in train_en]
        flattened_list_en = [item for sublist in tokenized_corpus_en for item in sublist]
        vocab_en = set(flattened_list_en)

        num_merges = 10000
        with open('./PhoMT/BPE/bp_en.txt', 'w+', encoding='utf-8') as f:
            learn_bpe(vocab_en, f, num_symbols=num_merges)

        # train vietnamese BPE
        tokenized_corpus_vi = [gensim.utils.simple_preprocess(text) for text in train_vi]
        flattened_list_vi = [item for sublist in tokenized_corpus_vi for item in sublist]
        vocab_vi = set(flattened_list_vi)
        
        num_merges = 10000
        with open('./PhoMT/BPE/bp_vi.txt', 'w+', encoding='utf-8') as f:
            learn_bpe(vocab_vi, f, num_symbols=num_merges)
        
        # create BPE vocabulary
        bp_en = BPE(open('./PhoMT/BPE/bp_en.txt', encoding='utf-8'))
        bp_vi = BPE(open('./PhoMT/BPE/bp_vi.txt', encoding='utf-8'))
        
        temp_vi, temp_en = [], []
        for vi, en in train_iter:
            temp_vi.append(bp_vi.segment_tokens(gensim.utils.simple_preprocess(vi)))
            temp_en.append(bp_en.segment_tokens(gensim.utils.simple_preprocess(en)))
            
        temp_vi = set([item for sublist in temp_vi for item in sublist])
        temp_en = set([item for sublist in temp_en for item in sublist])
        

    train_vi_counter = Counter(temp_vi)
    train_en_counter = Counter(temp_en)
        
    vi_vocab = getVocab(train_vi_counter)
    en_vocab = getVocab(train_en_counter)
    # torch.save((vi_vocab, en_vocab), 'vocab.pt')
    
    if not USE_BPE:
        tokenizer_src, tokenizer_tgt = tokenizer(), tokenizer()
    else:
        tokenizer_src, tokenizer_tgt = tokenizer_BPE(src=True), tokenizer_BPE(src=False)
        
    vocab_src, vocab_tgt = vi_vocab, en_vocab
    # vocab_src, vocab_tgt = torch.load('vocab.pt')
    
    return train_iter, val_iter, test_iter, vi_vocab, en_vocab, tokenizer_src, tokenizer_tgt

In [58]:
# train_iter, val_iter, test_iter, vocab_src, vocab_tgt, tokenizer_src, tokenizer_tgt = get_Vocab_Tokenizer(1000, 100, 100, USE_BPE=True)

In [59]:
# vocab_src.get_itos()

In [60]:
#     vi_train_path = './PhoMT/tokenization/train/train.vi'
#     en_train_path = './PhoMT/tokenization/train/train.en'

#     vi_val_path = './PhoMT/tokenization/dev/dev.vi'
#     en_val_path = './PhoMT/tokenization/dev/dev.en'

#     vi_test_path = './PhoMT/tokenization/test/test.vi'
#     en_test_path = './PhoMT/tokenization/test/test.en'
    
    
#     train_vi, train_en = [], []
#     with open(vi_train_path, encoding='utf-8') as f:
#         for line in f.readlines():
#             train_vi.append(line.replace('\n', ''))
#     with open(en_train_path, encoding='utf-8') as f:
#         for line in f.readlines():
#             train_en.append(line.replace('\n', ''))
        
#     val_vi, val_en = [], []
#     with open(vi_val_path, encoding='utf-8') as f:
#         for line in f.readlines():
#             val_vi.append(line.replace('\n', ''))
#     with open(en_val_path, encoding='utf-8') as f:
#         for line in f.readlines():
#             val_en.append(line.replace('\n', ''))     
    
#     test_vi, test_en = [], []
#     with open(vi_test_path, encoding='utf-8') as f:
#         for line in f.readlines():
#             test_vi.append(line.replace('\n', ''))
#     with open(en_test_path, encoding='utf-8') as f:
#         for line in f.readlines():
#             test_en.append(line.replace('\n', ''))  
            
#     train_vi, train_en = shuffle(train_vi, train_en)
#     val_vi, val_en = shuffle(val_vi, val_en)
#     test_vi, test_en = shuffle(test_vi, test_en)
    
#     max_train, max_val, max_test = 100,100,100
#     USE_BPE = True
    
#     train_vi, train_en = train_vi[:max_train], train_en[:max_train]
#     val_vi, val_en = val_vi[:max_val], val_en[:max_val]
#     test_vi, test_en = test_vi[:max_test], test_en[:max_test]
    

#     train_iter = list(zip(train_vi, train_en))
#     val_iter = list(zip(val_vi, val_en))
#     test_iter = list(zip(test_vi, test_en))
    
#     if not USE_BPE:
#         temp_vi, temp_en = [], []
#         for vi, en in train_iter:
#             for token in vi.split():
#                 temp_vi.append(token)
#             for token in en.split():
#                 temp_en.append(token)

#         train_vi_counter = Counter(temp_vi)
#         train_en_counter = Counter(temp_en)
#     else:
#         tokenized_corpus_en = [gensim.utils.simple_preprocess(text) for text in train_en]
#         flattened_list_en = [item for sublist in tokenized_corpus_en for item in sublist]
#         vocab_en = set(flattened_list_en)

#         num_merges = 10000
#         with open('./PhoMT/BPE/bp_en.txt', 'w+', encoding='utf-8') as f:
#             learn_bpe(vocab_en, f, num_symbols=num_merges)

#         # train vietnamese BPE
#         tokenized_corpus_vi = [gensim.utils.simple_preprocess(text) for text in train_vi]
#         flattened_list_vi = [item for sublist in tokenized_corpus_vi for item in sublist]
#         vocab_vi = set(flattened_list_vi)
        
#         num_merges = 10000
#         with open('./PhoMT/BPE/bp_vi.txt', 'w+', encoding='utf-8') as f:
#             learn_bpe(vocab_vi, f, num_symbols=num_merges)
        
#         # create BPE vocabulary
#         bp_en = BPE(open('./PhoMT/BPE/bp_en.txt', encoding='utf-8'))
#         bp_vi = BPE(open('./PhoMT/BPE/bp_vi.txt', encoding='utf-8'))
        
#         temp_vi, temp_en = [], []
#         for vi, en in train_iter:
#             temp_vi.append(bp_vi.segment_tokens(gensim.utils.simple_preprocess(vi)))
#             temp_en.append(bp_en.segment_tokens(gensim.utils.simple_preprocess(en)))
            
#         temp_vi = set([item for sublist in temp_vi for item in sublist])
#         temp_en = set([item for sublist in temp_en for item in sublist])
        

#     train_vi_counter = Counter(temp_vi)
#     train_en_counter = Counter(temp_en)
        
#     vi_vocab = getVocab(train_vi_counter)
#     en_vocab = getVocab(train_en_counter)
    
#     if not USE_BPE:
#         tokenizer_src, tokenizer_tgt = tokenizer(), tokenizer()
#     else:
#         tokenizer_src, tokenizer_tgt = tokenizer_BPE(src=True), tokenizer_BPE(src=False)
        
#     vocab_src, vocab_tgt = vi_vocab, en_vocab
    
#     return train_iter, val_iter, test_iter, vi_vocab, en_vocab, tokenizer_src, tokenizer_tgt

In [61]:
# train_en_counter

In [62]:
# vocab_src.get_itos()[3]


> Batching matters a ton for speed. We want to have very evenly
> divided batches, with absolutely minimal padding. To do this we
> have to hack a bit around the default torchtext batching. This
> code patches their default batching to make sure we search over
> enough sentences to find tight batches.

## Iterators

In [63]:
def collate_batch(
    batch,
    src_pipeline,
    tgt_pipeline,
    src_vocab,
    tgt_vocab,
    device,
    max_padding=128,
    pad_id=2,
):
    bs_id = torch.tensor([0], device=device)  # <s> token id
    eos_id = torch.tensor([1], device=device)  # </s> token id
    src_list, tgt_list = [], []
    for (_src, _tgt) in batch:
        processed_src = torch.cat(
            [
                bs_id,
                torch.tensor(
                    src_vocab(src_pipeline(_src)),
                    dtype=torch.int64,
                    device=device,
                ),
                eos_id,
            ],
            0,
        )
        processed_tgt = torch.cat(
            [
                bs_id,
                torch.tensor(
                    tgt_vocab(tgt_pipeline(_tgt)),
                    dtype=torch.int64,
                    device=device,
                ),
                eos_id,
            ],
            0,
        )
        src_list.append(
            # warning - overwrites values for negative values of padding - len
            pad(
                processed_src,
                (
                    0,
                    max_padding - len(processed_src),
                ),
                value=pad_id,
            )
        )
        tgt_list.append(
            pad(
                processed_tgt,
                (0, max_padding - len(processed_tgt)),
                value=pad_id,
            )
        )

    src = torch.stack(src_list)
    tgt = torch.stack(tgt_list)
    return (src, tgt)

In [64]:
def create_dataloaders(
    device,
    train_iter,
    val_iter,
    test_iter,
    vocab_src,
    vocab_tgt,
    tokenizer_src,
    tokenizer_tgt,
    batch_size,
    max_padding,
):
    def tokenize_de(text):
        return tokenize(text, tokenizer_src)

    def tokenize_en(text):
        return tokenize(text, tokenizer_tgt)

    def collate_fn(batch):
        return collate_batch(
            batch,
            tokenize_de,
            tokenize_en,
            vocab_src,
            vocab_tgt,
            device,
            max_padding=max_padding,
            pad_id=vocab_src.get_stoi()['<blank>'],
        )

    train_dataloader = DataLoader(
        train_iter,
        batch_size=batch_size,
        shuffle=True,
        collate_fn=collate_fn,
    )
    valid_dataloader = DataLoader(
        val_iter,
        batch_size=batch_size,
        shuffle=True,
        collate_fn=collate_fn,
    )
    
    test_dataloader = DataLoader(
        test_iter,
        batch_size=batch_size,
        shuffle=True,
        collate_fn=collate_fn,
    )

    return train_dataloader, valid_dataloader, test_dataloader

In [65]:
def visualizeLoss(train_loss, validation_loss, figure_path=None):
    plt.plot(train_loss, label='Train Loss', color='blue')
    plt.plot(validation_loss, label='Validation Loss', color='red')
    plt.xticks(range(len(train_loss)))
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.title('Loss Over Time')
    plt.legend()
#     plt.savefig(figure_path)
    plt.show()

In [66]:
# def saveData(train_loss, validation_loss, elapsed_time, config, params, json_path):
#     data = {}
#     data['train_loss'] = train_loss
#     data['validation_loss'] = validation_loss
#     data['elapsed_time'] = elapsed_time
#     data['config'] = config
#     data['params'] = params
#     with open(json_path, 'w') as f:
#         json.dump(data, f)

In [67]:
def saveData(gpu,train_iter,val_iter,test_iter,vocab_src,vocab_tgt,tokenizer_src,tokenizer_tgt,config,params,module,optimizer, lr_scheduler, train_epoch_loss,validation_epoch_loss,elapsed_time,model_path):
    data = {
        "gpu":gpu,
        "train_iter":train_iter,
        "val_iter":val_iter,
        "test_iter":test_iter,
        "vocab_src":vocab_src,
        "vocab_tgt":vocab_tgt,
        "tokenizer_src":tokenizer_src,
        "tokenizer_tgt":tokenizer_tgt,
        "config":config,
        "params":params,
        "model":module.state_dict(),
        "optimizer": optimizer.state_dict(),
        "lr_scheduler": lr_scheduler.state_dict(),
        "train_epoch_loss":train_epoch_loss,
        "validation_epoch_loss":validation_epoch_loss,
        "elapsed_time":elapsed_time
    }
    
    torch.save(data, model_path)

## Training the System

In [68]:
def train_model(
    gpu, train_iter, val_iter, test_iter, vocab_src, vocab_tgt, tokenizer_src, tokenizer_tgt, config, params
):
    print(f'Train worker process using GPU: {gpu} for training', flush=True)
    gc.collect()
    torch.cuda.set_device(gpu)

    pad_idx = vocab_tgt['<blank>']
    
    print_each = config['print_each']
    save_model = config['save_model']
    
    model_path = './models/' + config['file_prefix'] + '_' + params['attention_method'] + '.pt'
    
    len_tgt = params['tgt_dim']
    d_model = params['d_model']

    model = make_model(**params)

    model.cuda(gpu)
    module = model
    is_main_process = True

    criterion = LabelSmoothing(size=len_tgt, padding_idx=pad_idx, smoothing=0.1)
    criterion.cuda(gpu)

    train_dataloader, valid_dataloader, test_dataloader = create_dataloaders(
        gpu,
        train_iter,
        val_iter,
        test_iter,
        vocab_src,
        vocab_tgt,
        tokenizer_src,
        tokenizer_tgt,
        batch_size=config['batch_size'],
        max_padding=config['max_padding'],
    )

    optimizer = torch.optim.Adam(
        model.parameters(), lr=config['base_lr'], betas=(0.9, 0.98), eps=1e-9
    )

    lr_scheduler = LambdaLR(
        optimizer=optimizer,
        lr_lambda=lambda step: rate(step, d_model, factor=1, warmup=config['warmup']),
    )

    start_time = time.time()
    
    train_epoch_loss = []
    validation_epoch_loss = []
    
    for epoch in range(config['num_epochs']):
        module.train()
        print(f'[GPU{gpu}] Epoch {epoch} Training ====', flush=True)
        loss = run_epoch(
            (Batch(b[0], b[1], pad_idx) for b in train_dataloader),
            print_each,
            model,
            SimpleLossCompute(module.generator, criterion),
            optimizer,
            lr_scheduler,
            mode='train+log',
            accum_iter=config['accum_iter']
        )
        
        train_epoch_loss.append(loss)

        print('\n')

        GPUtil.showUtilization()
        gc.collect()
        torch.cuda.empty_cache()

        print('\n')

        print(f'[GPU{gpu}] Epoch {epoch} Validation ====', flush=True)
        module.eval()
        loss = run_epoch(
            (Batch(b[0], b[1], pad_idx) for b in valid_dataloader),
            print_each,
            model,
            SimpleLossCompute(module.generator, criterion),
            DummyOptimizer(),
            DummyScheduler(),
            mode='eval',
        )
        
        validation_epoch_loss.append(loss)
        
        gc.collect()
        torch.cuda.empty_cache()
        
        elapsed_time = round((time.time() - start_time) / 60, 2)
        print('Time Elapsed: {} \n'.format(elapsed_time))

        print(
            '--------------------------------------------------------------------------------------------------'
        )
    
        if save_model:
            model_path = './models/' + config['file_prefix'] + '_' + params['attention_method'] + '_' + str(epoch) + '.pt'
            
            saveData(
                gpu,train_iter,val_iter,
                test_iter,vocab_src,vocab_tgt,
                tokenizer_src,tokenizer_tgt,config,
                params,module, optimizer, lr_scheduler,
                train_epoch_loss,validation_epoch_loss, 
                elapsed_time, model_path)
            
        gc.collect()
        torch.cuda.empty_cache()

In [69]:
# max_train, max_val, max_test = 2977999, 18719, 19151
# # max_train, max_val, max_test = 50000, 15000, 15000
# # max_train, max_val, max_test = 1000, 500, 500
# train_iter, val_iter, test_iter, vocab_src, vocab_tgt, tokenizer_src, tokenizer_tgt = get_Vocab_Tokenizer(max_train, max_val, max_test, USE_BPE=True)

# config = {
#     'batch_size': 50,
#     'print_each': (len(train_iter) // 50 // 10), 
#     'num_epochs': 4,
#     'accum_iter': 10,
#     'base_lr': 1.0,
#     'max_padding': 15,
#     'warmup': 3000,
# }

# params = {
#     'src_dim': len(vocab_src),
#     'tgt_dim': len(vocab_tgt),
#     'N': 6,
#     'd_model': 512,
#     'd_ff': 2048,
#     'h': 8,
#     'dropout': 0.1
# }

In [70]:
# config['file_prefix'] = 'vi_to_en_BPE_complete'
# config['save_model'] = True

# for option in ['softmax', 'rela', 'sparsemax', 'entmax']:
# for option in ['softmax', 'rela', 'sparsemax']:
# # for option in ['softmax']:
#     gc.collect()
#     torch.cuda.empty_cache()
#     params['attention_method'] = option
    
#     train_model(
#         0,
#         train_iter,
#         val_iter,
#         test_iter,
#         vocab_src,
#         vocab_tgt,
#         tokenizer_src,
#         tokenizer_tgt,
#         config,
#         params
#     )
    
# # gc.collect()
# # torch.cuda.empty_cache()
# # params['attention_method'] = 'softmax'

# # train_model(
# #     0,
# #     train_iter,
# #     val_iter,
# #     test_iter,
# #     vocab_src,
# #     vocab_tgt,
# #     tokenizer_src,
# #     tokenizer_tgt,
# #     config,
# #     params
# # )

In [71]:
# softmax = torch.load('./models/vi_to_en_complete_softmax_0.pt')
# rela = torch.load('./models/vi_to_en_complete_rela_0.pt')
# sparsemax = torch.load('./models/vi_to_en_complete_sparsemax_0.pt')
# entmax = torch.load('./models/vi_to_en_complete_entmax_0.pt')

In [72]:
# plt.plot(softmax['train_epoch_loss'], label='Softmax Attention')
# plt.plot(rela['train_epoch_loss'], label='Rectified Linear Attention')
# plt.plot(sparsemax['train_epoch_loss'], label='Sparsemax Attention')
# plt.plot(entmax['train_epoch_loss'], label='Entmax Attenion')

# plt.xticks(range(len(softmax['train_epoch_loss'])))
# plt.xlabel('Epochs')
# plt.ylabel('Loss')
# plt.title('Train Loss Over Time')
# plt.legend()
# plt.show()

In [73]:
# plt.plot(softmax['validation_epoch_loss'], label='Softmax Attention')
# plt.plot(rela['validation_epoch_loss'], label='ReLa Attention')
# plt.plot(sparsemax['validation_epoch_loss'], label='Sparsemax Attention')
# plt.plot(entmax['validation_epoch_loss'], label='Entmax Attention')

# plt.xticks(range(len(softmax['validation_epoch_loss'])))
# plt.xlabel('Epochs')
# plt.ylabel('Loss')
# plt.title('Validation Loss Over Time')
# plt.legend()
# plt.show()

In [74]:
# fig, ax = plt.subplots(figsize=(5, 5))
# ax.bar(['softmax', 'rela', 'sparsemax', 'entmax'], [softmax['elapsed_time'],rela['elapsed_time'],sparsemax['elapsed_time'],entmax['elapsed_time']])
# ax.bar_label(ax.containers[0], label_type='edge')
# plt.ylabel('Minutes')
# # plt.legend()
# plt.show()


> Once trained we can decode the model to produce a set of
> translations. Here we simply translate the first sentence in the
> validation set. This dataset is pretty small so the translations
> with greedy search are reasonably accurate.

In [75]:
# def average(model, models):
#     'Average models into model'
#     for ps in zip(*[m.params() for m in [model] + models]):
#         ps[0].copy_(torch.sum(*ps[1:]) / len(ps[1:]))

# Results

On the WMT 2014 English-to-German translation task, the big
transformer model (Transformer (big) in Table 2) outperforms the
best previously reported models (including ensembles) by more than
2.0 BLEU, establishing a new state-of-the-art BLEU score of
28.4. The configuration of this model is listed in the bottom line
of Table 3. Training took 3.5 days on 8 P100 GPUs. Even our base
model surpasses all previously published models and ensembles, at a
fraction of the training cost of any of the competitive models.

On the WMT 2014 English-to-French translation task, our big model
achieves a BLEU score of 41.0, outperforming all of the previously
published single models, at less than 1/4 the training cost of the
previous state-of-the-art model. The Transformer (big) model trained
for English-to-French used dropout rate Pdrop = 0.1, instead of 0.3.


![](images/results.png)



> With the addtional extensions in the last section, the OpenNMT-py
> replication gets to 26.9 on EN-DE WMT. Here I have loaded in those
> parameters to our reimplemenation.

In [76]:
# x = torch.load('./metrics/vi_to_en_complete_softmax_3_metrics.pt')

In [77]:
# y = torch.load('./models/vi_to_en_complete_softmax_3.pt')

In [78]:
# for idx, (vi, en) in enumerate(y['test_iter']):
#     print(vi)
#     print(en)
#     print(x['tgt_truth'][idx])
#     break

In [79]:
# x['tgt_truth'] in y['test_iter']:

In [80]:
# y['test_iter']

In [81]:
# Load data and model for output checks

In [82]:
# model_path = './models/' + config['file_prefix'] + '_' + params['attention_method'] + '.pt'
# model = make_model(**params)
# model.load_state_dict(torch.load(model_path)['model'])

In [83]:
def joinBPE(sentence):
    final_sentence = []
    temp = ''
    for token in sentence:
        if '@@' in token:
            temp+=token.replace('@@','')
        else:
            temp+=token
            final_sentence.append(temp)
            temp = ''
    if temp:
        final_sentence.append(temp)
    
    return final_sentence

In [84]:
def check_outputs(
    valid_dataloader,
    model,
    vocab_src,
    vocab_tgt,
    n_examples=1,
    USE_BPE=False,
    pad_idx=2,
    eos_string='</s>',
):
    results = [()] * n_examples
    tgt_truth, tgt_pred = [], []
    print_each = (n_examples // 4) 
    
    for idx, (src, tgt) in enumerate(iter(valid_dataloader)):
        
#         if n_examples>1 and idx % print_each == 0:
#             print('Decoding {}% Complete ...'.format((idx/print_each)*25))
        
        if idx == n_examples: 
            break

        rb = Batch(src, tgt, pad_idx)

        greedy_decode(model, rb.src, rb.src_mask, 64, 0)[0]

        src_tokens = [vocab_src.get_itos()[x] for x in rb.src[0] if x != pad_idx]
        tgt_tokens = [vocab_tgt.get_itos()[x] for x in rb.tgt[0] if x != pad_idx]
        
        model_out = greedy_decode(model, rb.src, rb.src_mask, 72, 0)[0]
        model_txt = (
            ' '.join(
                [vocab_tgt.get_itos()[x] for x in model_out if x != pad_idx]
            ).split(eos_string, 1)[0]
            + eos_string
        )
        
        if not USE_BPE:
            tgt_pred.append(model_txt.replace('\n', '').split()[1:-2])
            tgt_truth.append([' '.join(tgt_tokens).replace('\n', '').split()[1:-2]])
        else:
            tgt_pred.append(joinBPE(model_txt.replace('\n', '').split()[1:-2]))
            tgt_truth.append([joinBPE(' '.join(tgt_tokens).replace('\n', '').split()[1:-2])])
        
        results[idx] = (rb, src_tokens, tgt_tokens, model_out, model_txt)
    
    score = bleu_score(tgt_pred, tgt_truth, max_n=4, weights=[0.25, 0.25, 0.25, 0.25])

    return model, tgt_truth, tgt_pred, score, results


def run_model_example(
    USE_BPE,
    n_examples, 
    device,
    train_iter,
    val_iter,
    test_iter,
    vocab_src,
    vocab_tgt,
    tokenizer_src,
    tokenizer_tgt,
    config,
    params,
    model_path
):

    _, _, test_dataloader = create_dataloaders(
        torch.device(0),
        train_iter,
        val_iter,
        test_iter,
        vocab_src,
        vocab_tgt,
        tokenizer_src,
        tokenizer_tgt,
        batch_size=1,
        max_padding=25,
    )

    model = make_model(**params)

    model.load_state_dict(
        torch.load(model_path, map_location=torch.device(0))['model']
    )
    
    model = model.to('cuda')
    model.eval()
    

    model, tgt_truth, tgt_pred, score, results = check_outputs(
        test_dataloader, model, vocab_src, vocab_tgt, n_examples=n_examples, USE_BPE=USE_BPE
    )
    
    print('\nComplete.')
    
    return model, tgt_truth, tgt_pred, score, results

In [99]:
start_time = time.time()
gc.collect()
torch.cuda.empty_cache()
# model_path = './models/' + config['file_prefix'] + '_' + params['attention_method'] + '_' + str(config['num_epochs']-1) + '.pt'
# for model_name in ['vi_to_en_BPE_complete_softmax_3.pt']:
for model_name in ['vi_to_en_complete_softmax_3.pt', 'vi_to_en_complete_rela_3.pt', 'vi_to_en_complete_sparsemax_3.pt']:
    model_path = './models/'+ model_name
    start_time = time.time()
    gc.collect()
    torch.cuda.empty_cache()
    
    temp = [('Hôm nay là một ngày đẹp trời', 'Today is a beautiful day')] 

    model_data = torch.load(model_path)
    model, tgt_truth, tgt_pred, score, results = run_model_example(
        False,
        1, 
        0,
        model_data['train_iter'],
        model_data['val_iter'],
        temp,
#         model_data['test_iter'],
        model_data['vocab_src'],
        model_data['vocab_tgt'],
        model_data['tokenizer_src'],
        model_data['tokenizer_tgt'],
        model_data['config'],
        model_data['params'],
        model_path
    )

    elapsed_time = round((time.time()-start_time)/60, 2)
    print('Completion Time:', elapsed_time)
    print('Score:', score)
    
    
    data = {
        "tgt_truth": tgt_truth,
        "tgt_pred": tgt_pred,
        "score": score,
        "results": results,
        "elapsed_time": elapsed_time
    }
    
    print(model_name)
    print(data['tgt_truth'])
    print(data['tgt_pred'])
    
#     metric_path = './metrics_test/' + model_name.replace('.pt','') + '_metrics' + '.pt'
#     torch.save(data, metric_path)
    
    start_time = time.time()


Complete.
Completion Time: 0.26
Score: 0.0
vi_to_en_complete_softmax_3.pt
[[['Today', 'is', 'a', 'beautiful']]]
[['It', "'s", 'a', 'beautiful', 'day']]


KeyboardInterrupt: 

## Attention Visualization

> Even with a greedy decoder the translation looks pretty good. We
> can further visualize it to see what is happening at each layer of
> the attention

In [85]:
def mtx2df(m, max_row, max_col, row_tokens, col_tokens):
    'convert a dense matrix to a data frame with row and column indices'
    return pd.DataFrame(
        [
            (
                r,
                c,
                float(m[r, c]),
                '%.3d %s' % (r, row_tokens[r] if len(row_tokens) > r else '<blank>'),
                '%.3d %s' % (c, col_tokens[c] if len(col_tokens) > c else '<blank>'),
            )
            for r in range(m.shape[0])
            for c in range(m.shape[1])
            if r < max_row and c < max_col
        ],
        # if float(m[r,c]) != 0 and r < max_row and c < max_col],
        columns=['row', 'column', 'value', 'row_token', 'col_token'],
    )


def attn_map(attn, layer, head, row_tokens, col_tokens, max_dim=30):
    df = mtx2df(
        attn[0, head].data,
        max_dim,
        max_dim,
        row_tokens,
        col_tokens,
    )
    return (
        alt.Chart(data=df)
        .mark_rect()
        .encode(
            x=alt.X('col_token', axis=alt.Axis(title='')),
            y=alt.Y('row_token', axis=alt.Axis(title='')),
            color=alt.Color('value', scale=alt.Scale(scheme='lightgreyteal')),
            
            
            tooltip=['row', 'column', 'value', 'row_token', 'col_token'],
        )
        .properties(height=400, width=400)
        .interactive()
    )

In [86]:
def get_encoder(model, layer):
    return model.encoder.layers[layer].self_attn.attn


def get_decoder_self(model, layer):
    return model.decoder.layers[layer].self_attn.attn


def get_decoder_src(model, layer):
    return model.decoder.layers[layer].src_attn.attn


def visualize_layer(model, layer, getter_fn, ntokens, row_tokens, col_tokens):
    # ntokens = last_example[0].ntokens
    attn = getter_fn(model, layer)
    n_heads = attn.shape[1]
    charts = [
        attn_map(
            attn,
            0,
            h,
            row_tokens=row_tokens,
            col_tokens=col_tokens,
            max_dim=ntokens,
        )
        for h in range(n_heads)
    ]
    assert n_heads == 8
    return alt.vconcat(
        charts[0]
        # | charts[1]
        | charts[2]
        # | charts[3]
        | charts[4]
        # | charts[5]
        | charts[6]
        # | charts[7]
        # layer + 1 due to 0-indexing
    ).properties(title='Layer %d' % (layer + 1))

## Encoder Self Attention

In [98]:
# ['vi_to_en_complete_softmax_3.pt', 'vi_to_en_complete_rela_3.pt', 'vi_to_en_complete_sparsemax_3.pt']

In [107]:
def viz_encoder_self():
    for model_name in ['vi_to_en_complete_rela_3.pt']:
        model_path = './models/'+ model_name
        start_time = time.time()
        gc.collect()
        torch.cuda.empty_cache()
    
        temp = [('Hôm nay là một ngày đẹp trời', 'Today is a beautiful day')] 

        model_data = torch.load(model_path)
        model, tgt_truth, tgt_pred, score, results = run_model_example(
            False,
            1, 
            0,
            model_data['train_iter'],
            model_data['val_iter'],
            temp,
    #         model_data['test_iter'],
            model_data['vocab_src'],
            model_data['vocab_tgt'],
            model_data['tokenizer_src'],
            model_data['tokenizer_tgt'],
            model_data['config'],
            model_data['params'],
            model_path
        )
    
    
        example = results[len(results) - 1]  # batch object for the final example

        layer_viz = [
            visualize_layer(
                model, layer, get_encoder, len(example[1]), example[1], example[1]
            )
            for layer in range(6)
        ]
        return alt.hconcat(
            layer_viz[0]
            & layer_viz[1]
                    & layer_viz[2]
            & layer_viz[3]
                    & layer_viz[4]
            & 
            layer_viz[5]
        )


show_example(viz_encoder_self)


Complete.


alt.HConcatChart(...)

## Decoder Self Attention

In [90]:
def viz_decoder_self():
    
    for model_name in ['vi_to_en_complete_softmax_3.pt', 'vi_to_en_complete_rela_3.pt', 'vi_to_en_complete_sparsemax_3.pt']:
        model_path = './models/'+ model_name
        start_time = time.time()
        gc.collect()
        torch.cuda.empty_cache()
    
        temp = [('Hôm nay là một ngày đẹp trời', 'Today is a beautiful day')] 

        model_data = torch.load(model_path)
        model, tgt_truth, tgt_pred, score, results = run_model_example(
            False,
            1, 
            0,
            model_data['train_iter'],
            model_data['val_iter'],
            temp,
    #         model_data['test_iter'],
            model_data['vocab_src'],
            model_data['vocab_tgt'],
            model_data['tokenizer_src'],
            model_data['tokenizer_tgt'],
            model_data['config'],
            model_data['params'],
            model_path,
        )

    
        example = results[len(results) - 1]  # batch object for the final example

        layer_viz = [
            visualize_layer(
                model,
                layer,
                get_decoder_self,
                len(example[1]),
                example[1],
                example[1],
            )
            for layer in range(6)
        ]
        return alt.hconcat(
            layer_viz[0]
    #         & layer_viz[1]
    #         & layer_viz[2]
            & layer_viz[3]
    #         & layer_viz[4]
            & layer_viz[5]
        )


    show_example(viz_decoder_self)


Complete.


alt.HConcatChart(...)

## Decoder Src Attention

In [ ]:
'vi_to_en_complete_softmax_3.pt', 'vi_to_en_complete_rela_3.pt', 'vi_to_en_complete_sparsemax_3.pt'

In [95]:
def viz_decoder_src():
    for model_name in ['vi_to_en_complete_softmax_3.pt']:
        model_path = './models/'+ model_name
        start_time = time.time()
        gc.collect()
        torch.cuda.empty_cache()
    
        temp = [('Hôm nay là một ngày đẹp trời', 'Today is a beautiful day')] 

        model_data = torch.load(model_path)
        model, tgt_truth, tgt_pred, score, results = run_model_example(
            False,
            1, 
            0,
            model_data['train_iter'],
            model_data['val_iter'],
            temp,
    #         model_data['test_iter'],
            model_data['vocab_src'],
            model_data['vocab_tgt'],
            model_data['tokenizer_src'],
            model_data['tokenizer_tgt'],
            model_data['config'],
            model_data['params'],
            model_path
        )
        
        example = results[len(results) - 1]  # batch object for the final example

        layer_viz = [
            visualize_layer(
                model,
                layer,
                get_decoder_src,
                max(len(example[1]), len(example[2])),
                example[1],
                example[2],
            )
            for layer in range(6)
        ]
        return alt.hconcat(
#             layer_viz[0]
#             & layer_viz[1]
#             & layer_viz[2]
#             & layer_viz[3]
#             & layer_viz[4]
#             & 
            layer_viz[5]
        )


show_example(viz_decoder_src)


Complete.


alt.HConcatChart(...)

# Conclusion

 Hopefully this code is useful for future research. Please reach
 out if you have any issues.


 Cheers,
 Sasha Rush, Austin Huang, Suraj Subramanian, Jonathan Sum, Khalid Almubarak,
 Stella Biderman